In [3]:
# import the kaggle.json from kaggle API into colab
# do this command

# install kaggle library
!pip install kaggle
# make a directory named .kaggle
!mkdir ~/.kaggle
# copy the kaggle.json into this new directory
!cp kaggle.json ~/.kaggle/
# alocate the required permission for this file
!chmod 600 ~/.kaggle/kaggle.json
# download dataset
!kaggle datasets download shivam2503/diamonds

mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/733k [00:00<?, ?B/s]
100% 733k/733k [00:00<00:00, 127MB/s]


In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [5]:
url = 'https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv'
df = pd.read_csv(url)

In [7]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

34521 train examples
8631 validation examples
10788 test examples


In [8]:
# create target column
# 0 = low, 1 = high
df['target'] = np.where(df['price'] <= 1000, 0, 1)
# drop un-used columns
df = df.drop(columns=['price'])

In [9]:
# create dataset tf.data from pandas dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = df.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size = len(dataframe))
  ds = ds.batch(batch_size)
  return ds

batch_size = 10 #small batch size for demonstration
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [11]:
example_batch = next(iter(train_ds))[0]
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [12]:
# numeric columns
carat = feature_column.numeric_column('carat')
demo(carat)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[[0.7 ]
 [0.54]
 [0.41]
 [0.43]
 [0.32]
 [0.31]
 [1.47]
 [0.26]
 [0.4 ]
 [0.53]]


In [13]:
# bucketized column
carat = feature_column.numeric_column('carat')
carat_buckets = feature_column.bucketized_column(carat, boundaries=[1,2])
demo(carat_buckets)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [14]:
# categorical
color_type = feature_column.categorical_column_with_vocabulary_list(
    'color', ['E','I', 'J', 'D', 'H', 'G', 'F']
)

color_type_one_hot = feature_column.indicator_column(color_type)
demo(color_type_one_hot)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[[1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]]


In [15]:
# Embedding
clarity = feature_column.categorical_column_with_vocabulary_list(
    'clarity', df.clarity.unique()
)

clarity_embedding = feature_column.embedding_column(clarity, dimension=6)
demo(clarity_embedding)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[[ 0.62880474 -0.47700405 -0.33082196 -0.30582657 -0.13923575  0.7310454 ]
 [ 0.072032    0.6628276   0.3250774   0.5502012  -0.11018097  0.1510863 ]
 [ 0.32835105  0.7354041   0.09370571 -0.12796547  0.22361277 -0.13337669]
 [-0.47970665  0.34657586 -0.4382733  -0.04013623 -0.06052717 -0.509754  ]
 [ 0.62880474 -0.47700405 -0.33082196 -0.30582657 -0.13923575  0.7310454 ]
 [ 0.62880474 -0.47700405 -0.33082196 -0.30582657 -0.13923575  0.7310454 ]
 [ 0.32835105  0.7354041   0.09370571 -0.12796547  0.22361277 -0.13337669]
 [-0.8130617   0.41242817 -0.4093392  -0.22138116 -0.02640382  0.23072772]
 [ 0.62880474 -0.47700405 -0.33082196 -0.30582657 -0.13923575  0.7310454 ]
 [ 0.38724998 -0.22759522 -0.03395327  0.22571841 -0.01494894 -0.07700925]]


In [16]:
# hashed feature
clarity_hashed = feature_column.categorical_column_with_hash_bucket(
    'clarity', hash_bucket_size=5
)
demo(feature_column.indicator_column(clarity_hashed))

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[[1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]


In [17]:
# crossed feature
# crossed data should be string, categorical, or bucketized
crossed_feature = feature_column.crossed_column([carat_buckets, color_type], hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

Instructions for updating:
Use `tf.keras.layers.experimental.preprocessing.HashedCrossing` instead for feature crossing when preprocessing data to train a Keras model.


[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [18]:
# pick which feature column to use
feature_columns = []

# numeric columns
for header in ['carat', 'depth', 'x', 'y', 'z']:
  feature_columns.append(feature_column.numeric_column(header))

# create feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [19]:
model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(128, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dropout(.1),
    layers.Dense(1)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.fit(train_ds,validation_data = val_ds, epochs=10)

Epoch 1/10
1686/1686 [==============================] - 9s 4ms/step - loss: 0.2070 - accuracy: 0.9099 - val_loss: 0.1479 - val_accuracy: 0.9365
Epoch 2/10
1686/1686 [==============================] - 9s 5ms/step - loss: 0.1597 - accuracy: 0.9333 - val_loss: 0.2269 - val_accuracy: 0.8926
Epoch 3/10
1686/1686 [==============================] - 7s 4ms/step - loss: 0.1576 - accuracy: 0.9338 - val_loss: 0.1427 - val_accuracy: 0.9379
Epoch 4/10
1686/1686 [==============================] - 8s 4ms/step - loss: 0.1525 - accuracy: 0.9350 - val_loss: 0.1428 - val_accuracy: 0.9373
Epoch 5/10
1686/1686 [==============================] - 9s 5ms/step - loss: 0.1498 - accuracy: 0.9367 - val_loss: 0.1434 - val_accuracy: 0.9387
Epoch 6/10
1686/1686 [==============================] - 7s 4ms/step - loss: 0.1493 - accuracy: 0.9361 - val_loss: 0.1555 - val_accuracy: 0.9383
Epoch 7/10
1686/1686 [==============================] - 8s 5ms/step - loss: 0.1469 - accuracy: 0.9371 - val_loss: 0.1494 - val_accuracy: